# CHAPTER 40 - METACLASSES

## To Metaclass or Not to Metaclass

In [52]:
type([])

list

In [53]:
type(type([]))

type

In [54]:
class C: ...
X = C()

In [55]:
type(X)

__main__.C

In [56]:
X.__class__

__main__.C

In [57]:
type(C)

type

In [58]:
C.__class__

type

In [59]:
class Eggs: ...

In [60]:
class Spam(Eggs):
    data = 1
    def meth(self, arg):
        return self.data + arg

In [61]:
x = Spam = type("Spam", (), {"data": 1, "meth" : (lambda x, y: x.data + y)})

In [62]:
i = x()

In [63]:
x, i

(__main__.Spam, <__main__.Spam at 0x7f4590c30640>)

In [64]:
i.data, i.meth(2)

(1, 3)

In [65]:
x.__bases__

(object,)

In [66]:
[(a, v) for a, v in x.__dict__.items() if not a.startswith("__")]

[('data', 1), ('meth', <function __main__.<lambda>(x, y)>)]

In [67]:
class Meta(type):
    def __new__(meta, classname, supers, classdict):
        # Run by inherited type.__call__
        return type.__new__(meta, classname, supers, classdict)

### A BASIC METACLASS

In [68]:
class Meta(type):
    def __new__(meta, classname, supers, classdict):
        # Run by inherited type.__call__
        return type.__new__(meta, classname, supers, classdict)

In [69]:
class MetaOne(type):
    def __new__(meta, classname, supers, classdict):
        print('In MetaOne.new:', meta, classname, supers, classdict, sep='\n...')
        return type.__new__(meta, classname, supers, classdict)

In [70]:
class Eggs: ...

In [71]:
class Spam(Eggs, metaclass=MetaOne):
    data = 1
    def meth(self, arg):
        return self.data + arg

In MetaOne.new:
...<class '__main__.MetaOne'>
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x7f4590c9c430>}


In [72]:
X = Spam()

In [73]:
X.data, X.meth(2)

(1, 3)

### CUSTOMIZING CONSTRUCTON AND INITALIZATION

In [74]:
class MetaTwo(type):
    def __new__(meta, classname, supers, classdict):
        print('In MetaTwo.new:', meta, classname, supers, classdict, sep='\n...')
        return type.__new__(meta, classname, supers, classdict)

    def __init__(meta, classname, supers, classdict):
        print('In MetaTwo.init:', meta, classname, supers, classdict, sep='\n...')
        return type.__init__(meta, classname, supers, classdict)

In [75]:
class Eggs: ...

In [76]:
class Spam(Eggs, metaclass=MetaTwo):
    data = 1
    def meth(self, arg):
        return self.data + arg

In MetaTwo.new:
...<class '__main__.MetaTwo'>
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x7f4590c9cd30>}
In MetaTwo.init:
...<class '__main__.Spam'>
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x7f4590c9cd30>}


In [77]:
def MetaFunction(classname, supers, classdict):
    print('In MetaFunction:', classname, supers, classdict, sep='\n...')
    return type(classname, supers, classdict)

In [78]:
class Eggs: ...

In [79]:
class Spam(Eggs, metaclass=MetaFunction):
    data = 1
    def meth(self, arg):
        return self.data + arg

In MetaFunction:
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x7f4590c9c1f0>}


In [80]:
X = Spam()

In [81]:
X.data, X.meth(2)

(1, 3)

In [89]:
class MetaObj:
    def __call__(self, classname, supers, classdict):
        print('In MetaObj.__call__:', classname, supers, classdict, sep='\n...')
        Class = self.__New__(classname, supers, classdict)
        self.Init__(Class, classname, supers, classdict)
        return Class
    
    def __New__(self, classname, supers, classdict):
        print('In MetaObj.__New__:', classname, supers, classdict, sep='\n...')
        return type.__new__(self, classname, supers, classdict)

    def __Init__(self, Class, classname, supers, classdict):
        print('In MetaObj.init:', classname, supers, classdict, sep='\n...')
        print('...init class object:', list(Class.__dict__.keys()))

In [90]:
class Eggs: ...

In [91]:
class Spam(Eggs, metaclass=MetaObj()):
    data = 1
    def meth(self, arg):
        return self.data + arg

In MetaObj.__call__:
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x7f4590cb8ca0>}
In MetaObj.__New__:
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x7f4590cb8ca0>}


TypeError: type.__new__(X): X is not a type object (MetaObj)

### OVERLOADGING CLASS CREATION CALLS WITH METACLASSES

In [2]:
class SuperMeta(type):
    def __call__(meta, classname, supers, classdict):
        print('In SuperMeta.call:', classname, supers, classdict, sep='\n...')
        return type.__call__(meta, classname, supers, classdict)

    def __init__(Class, classname, supers, classdict):
        print('In SuperMeta.init:', classname, supers, classdict, sep='\n...')
        print('...init class object:', list(Class.__dict__.keys()))

In [4]:
class SubMeta(type, metaclass=SuperMeta):
    def __new__(meta, classname, supers, classdict):
        print('In SubMeta.new:', classname, supers, classdict, sep='\n...')
        return type.__new__(meta, classname, supers, classdict)

    def __init__(Class, classname, supers, classdict):
        print('In SubMeta.init:', classname, supers, classdict, sep='\n...')
        print('...init class object:', list(Class.__dict__.keys()))

In SuperMeta.init:
...SubMeta
...(<class 'type'>,)
...{'__module__': '__main__', '__qualname__': 'SubMeta', '__new__': <function SubMeta.__new__ at 0x00000219B5ED17E0>, '__init__': <function SubMeta.__init__ at 0x00000219B5ED1360>}
...init class object: ['__module__', '__new__', '__init__', '__doc__']


In [5]:
class Eggs: ...

In [6]:
class Spam(Eggs, metaclass=SubMeta):
    data = 1
    def meth(self, arg):
        return self.data + arg

In SuperMeta.call:
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x00000219B5ED2320>}
In SubMeta.new:
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x00000219B5ED2320>}
In SubMeta.init:
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x00000219B5ED2320>}
...init class object: ['__module__', 'data', 'meth', '__doc__']


In [7]:
class MetaOne(type):
    def __new__(meta, classname, supers, classdict):
        print('In MetaOne.new:', meta, classname, supers, classdict, sep='\n...')
        return type.__new__(meta, classname, supers, classdict)

    def toast(self):
        return 'toast'

In [8]:
class Super(metaclass=MetaOne):
    def spam(self):
        return 'spam'

In MetaOne.new:
...<class '__main__.MetaOne'>
...Super
...()
...{'__module__': '__main__', '__qualname__': 'Super', 'spam': <function Super.spam at 0x00000219B5ED2B00>}


In [9]:
class Sub(Super):
    def eggs(self):
        return 'eggs'

In MetaOne.new:
...<class '__main__.MetaOne'>
...Sub
...(<class '__main__.Super'>,)
...{'__module__': '__main__', '__qualname__': 'Sub', 'eggs': <function Sub.eggs at 0x00000219B5ED29E0>}


In [11]:
X = Sub()

In [12]:
X.eggs()

'eggs'

In [13]:
X.spam()

'spam'

In [14]:
Sub.eggs(X)

'eggs'

In [15]:
Sub.spam(X)

'spam'

In [16]:
X.toast()

AttributeError: 'Sub' object has no attribute 'toast'

In [17]:
Sub.toast(X)

TypeError: MetaOne.toast() takes 1 positional argument but 2 were given

In [18]:
Sub.toast

<bound method MetaOne.toast of <class '__main__.Sub'>>

In [19]:
Sub.spam

<function __main__.Super.spam(self)>

### METACLASSES VERSUS SUPERCLASSES

In [20]:
class A(type): attr = 1

In [21]:
class B(metaclass=A): pass

In [22]:
I = B()

In [23]:
B.attr

1

In [24]:
I.attr

AttributeError: 'B' object has no attribute 'attr'

In [25]:
class A: attr = 1

In [26]:
class B(A): pass

In [27]:
I = B()

In [28]:
I.attr

1

In [29]:
I.attr

1

In [31]:
'attr' in B.__dict__, 'attr' in A.__dict__

(False, True)

### METACLASS METHODS

In [6]:
class A(type):
    def x(cls):
        print("ax", cls)

    def y(cls):
        print("ay", cls)

In [7]:
class B(metaclass=A):
    def y(self):
        print("by", self)

    def z(self):
        print("bz", self)

In [10]:
B.x

<bound method A.x of <class '__main__.B'>>

In [9]:
B.y

<function __main__.B.y(self)>

In [11]:
B.z

<function __main__.B.z(self)>

In [12]:
B.x()

ax <class '__main__.B'>


In [13]:
I = B()

In [15]:
I.y()

by <__main__.B object at 0x7fbd5029cac0>


In [16]:
I.z()

bz <__main__.B object at 0x7fbd5029cac0>


In [17]:
I.x()

AttributeError: 'B' object has no attribute 'x'

### METACLASS METHODS VERSUS CLASS METHODS

In [18]:
class A(type):
    def a(cls):
        cls.x = cls.y + cls.z

In [20]:
class B(metaclass=A):
    y, z = 11, 22

    @classmethod
    def b(cls):
        return cls.x

In [21]:
B.a()

In [23]:
B.x

33

In [24]:
I = B()

In [25]:
I.x, I.y, I.z

(33, 11, 22)

In [26]:
I.b()

33

In [27]:
I.a()

AttributeError: 'B' object has no attribute 'a'

### OPERATOR OVERLOADING IN METACLASS METHODS

In [29]:
class A(type):
    def __getitem__(cls, i):
        return cls.data[i]

In [30]:
class B(metaclass=A):
    data = "spam"

In [31]:
B[0]

's'

In [32]:
B.__getitem__

<bound method A.__getitem__ of <class '__main__.B'>>

In [33]:
I = B()

In [34]:
I.data, B.data

('spam', 'spam')

In [35]:
I[0]

TypeError: 'B' object is not subscriptable

In [36]:
class A(type):
    def __getattr__(cls, name):
        return getattr(cls.data, name)

In [37]:
class B(metaclass=A):
    data = "spamm"

In [38]:

B.upper()

'SPAMM'

In [39]:
B.upper

<function str.upper()>

In [40]:
B.__getattr__

<bound method A.__getattr__ of <class '__main__.B'>>

In [41]:
I = B()

In [42]:
I.upper

AttributeError: 'B' object has no attribute 'upper'

In [43]:
I.__getattr__

AttributeError: 'B' object has no attribute '__getattr__'

### EXAMPLE: ADDING METHODS TO CLASSES

In [1]:
class Client1:
    def __init__(self, value):
        self.value = value
    def spam(self):
        return self.value * 2

In [8]:
class Client2:
    value = "ni?"

In [9]:
def eggsfunc(obj):
    return obj.value * 4

In [10]:
def hamfunc(obj, value):
    return value + "ham"

In [14]:
Client1.eggs = eggsfunc
Client1.ham = hamfunc

In [15]:
Client2.eggs = eggsfunc
Client2.ham = hamfunc

In [16]:
X = Client1("Ni!")

In [17]:
X.spam()

'Ni!Ni!'

In [18]:
X.eggs()

'Ni!Ni!Ni!Ni!'

In [19]:
X.ham("bacon")

'baconham'

In [20]:
Y = Client2()

In [21]:
Y.eggs()

'ni?ni?ni?ni?'

In [22]:
Y.ham("bacon")

'baconham'

### METACLASS-BASSED AUGMENTATIONS

In [28]:
def eggsfunc(obj):
    return obj.value * 4

In [29]:
def hamfunc(obj, value):
    return value + "ham"

In [30]:
class Extender(type):
    def __new__(meta, classname, supers, classdict):
        classdict["eggs"] = eggsfunc
        classdict["ham"] = hamfunc
        return type.__new__(meta, classname, supers, classdict)

In [31]:
class Client1(metaclass=Extender):
    def __init__(self, value):
        self.value = value
    def spam(self):
        return self.value * 2

In [32]:
class Client2(metaclass=Extender):
    value = "ni?"

In [33]:
X = Client1('Ni!')

In [35]:
X.spam()

'Ni!Ni!'

In [36]:
X.eggs()

'Ni!Ni!Ni!Ni!'

In [37]:
X.ham('bacon')

'baconham'

In [39]:
Y = Client2()

In [40]:
Y.eggs()

'ni?ni?ni?ni?'

In [41]:
Y.ham('bacon')

'baconham'

In [44]:
class MetaExtend(type):
    def __new__(meta, classname, supers, classdict):
        if sometest():
            classdict['eggs'] = eggsfunc1
        else:
            classdict['eggs'] = eggsfunc2
        if someothertest():
            classdict['ham'] = hamfunc
        else:
            classdict['ham'] = lambda *args: 'Not supported'
        return type.__new__(meta, classname, supers, classdict)

In [45]:
def eggsfunc(obj):
    return obj.value * 4

In [46]:
def hamfunc(obj, value):
    return value + "ham"

In [47]:
def Extender(aClass):
    aClass.eggs = eggsfunc
    aClass.ham = hamfunc
    return aClass

In [48]:
@Extender
class Client1:
    def __init__(self, value):
        self.value = value
    def spam(self):
        return self.value * 2

In [49]:
@Extender
class Client2:
    value = "ni?"

In [50]:
X.spam()

'Ni!Ni!'

In [51]:
X.eggs()

'Ni!Ni!Ni!Ni!'

In [54]:
X.ham('bacon')

'baconham'

In [56]:
Y  = Client2()

In [57]:
Y.eggs()

'ni?ni?ni?ni?'

In [58]:
Y.ham('bacon')

'baconham'

In [60]:
def Tracer(aClass):
    class Wrapper:
        def __init__(self, *args, **kargs):
            self.wrapped = aClass(*args, **kargs)
        def __getattr__(self, attrname):
            print('Trace:', attrname)
            return getattr(self.wrapped, attrname)
    return Wrapper

In [62]:
@Tracer
class Person:
    def __init__(self, name, hours, rate):
        self.name = name
        self.hours = hours
        self.rate = rate
    def pay(self):
        return self.hours * self.rate

In [67]:
bob = Person('Bob', 40, 50)

In [68]:
print(bob.name)

Trace: name
Bob


In [69]:
print(bob.pay())

Trace: pay
2000


In [72]:
def Tracer(classname, supers, classdict):
    aClass = type(classname, supers, classdict)
    class Wrapper:
        def __init__(self, *args, **kargs):
            self.wrapped = aClass(*args, **kargs)
        def __getattr__(self, attrname):
            print('Trace:', attrname)
            return getattr(self.wrapped, attrname)
    return Wrapper

In [75]:
class Person(metaclass=Tracer):
    def __init__(self, name, hours, rate):
        self.name = name
        self.hours = hours
        self.rate = rate
    def pay(self):
        return self.hours * self.rate    

In [76]:
bob = Person('Bob', 40, 50)
bob = Person('Bob', 40, 50)
print(bob.pay())

Trace: pay
2000
